In [12]:
%%capture
!pip install -q langchain langchain-community faiss-cpu  pypdf langchain-text-splitters


In [13]:
import os
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [14]:
from google.colab import files

uploaded = files.upload()


Saving yolo.pdf to yolo.pdf


In [15]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader

file_name = list(uploaded.keys())[0]

if file_name.endswith(".pdf"):
    loader = PyPDFLoader(file_name)
else:
    loader = TextLoader(file_name)

documents = loader.load()

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

len(chunks)

106

In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

In [18]:
#gpt-4.1-nano

In [19]:
!pip install -qU langchain-community faiss-cpu

In [20]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)


In [21]:
#vectorstore.index_to_docstore_id

In [22]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [23]:
retriever.invoke('who is the pm of india')

[Document(id='81d2977c-088d-42ea-9cdc-65e6224a7198', metadata={'producer': 'pdfTeX-1.40.12', 'creator': 'LaTeX with hyperref package', 'creationdate': '2016-05-11T00:04:54+00:00', 'author': '', 'keywords': '', 'moddate': '2016-05-11T00:04:54+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.1415926-2.3-1.40.12 (TeX Live 2011) kpathsea version 6.0.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'yolo.pdf', 'total_pages': 10, 'page': 3, 'page_label': '4'}, page_content='classify regions, predict bounding boxes for high scoring\nregions, etc. Our system replaces all of these disparate parts\nwith a single convolutional neural network. The network\nperforms feature extraction, bounding box prediction, non-\nmaximal suppression, and contextual reasoning all concur-\nrently. Instead of static features, the network trains the fea-\ntures in-line and optimizes them for the detection task. Our\nuniﬁed architecture leads to a faster, more accurate model\nthan DPM.'),
 Document

In [24]:
%%capture
!pip install -U langchain-openai

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5-nano")

In [28]:
from langchain_core.prompts import PromptTemplate

In [29]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [31]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

'First, YOLO is extremely fast. Since we frame detection\nas a regression problem we don’t need a complex pipeline.\nWe simply run our neural network on a new image at test\ntime to predict detections. Our base network runs at 45\nframes per second with no batch processing on a Titan X\nGPU and a fast version runs at more than 150 fps. This\nmeans we can process streaming video in real-time with\nless than 25 milliseconds of latency. Furthermore, YOLO\n\nONR N00014-13-1-0720, NSF IIS-1338054, and The Allen\nDistinguished Investigator Award.\n\nVOC 2012 test mAP aero bike bird boat bottle bus car cat chair cow table dog horse mbike personplant sheep sofa train tv\nMR CNN MORE DATA [11] 73.9 85.5 82.9 76.6 57.8 62.7 79.4 77.2 86.6 55.0 79.1 62.2 87.0 83.4 84.7 78.9 45.3 73.4 65.8 80.3 74.0\nHyperNet VGG 71.4 84.2 78.5 73.6 55.6 53.7 78.7 79.8 87.7 49.6 74.9 52.1 86.0 81.7 83.3 81.8 48.6 73.5 59.4 79.9 65.7\nHyperNet SP 71.3 84.1 78.3 73.3 55.5 53.6 78.6 79.6 87.5 49.5 74.9 52.1 85.6 81.6

In [32]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [33]:
answer = llm.invoke(final_prompt)
print(answer.content)

No. The transcript discusses You Only Look Once (YOLO), a real-time object detection system, its speed (base network around 45 fps, fast version over 150 fps), predicting detections via regression, and real-time video latency under 25 ms, along with related references. Nuclear fusion is not mentioned.


In [34]:
def ask_q(question):
  retrieved_docs    = retriever.invoke(question)
  final_prompt = prompt.invoke({"context": context_text, "question": question})
  answer = llm.invoke(final_prompt)
  print(answer.content)

In [35]:
ask_q("explain yolo")

YOLO stands for You Only Look Once. It’s a unified, real-time approach to object detection. Rather than using classifiers to perform detection, YOLO frames object detection as a regression problem to predict spatially separated bounding boxes (in a single pass). At test time, you simply run a single neural network on a new image to predict detections, without a complex pipeline. It’s extremely fast: the base network runs at about 45 frames per second on a Titan X (no batch processing), and a fast version runs at over 150 fps, enabling real-time streaming video with latency under ~25 ms.


In [36]:
ask_q("who is the pm of india")

I don't know.
